In [1]:
import pandas as pd

In [2]:
dp = pd.read_csv('https://stepik.org/media/attachments/lesson/383523/lesson_3_data__1_.csv', encoding='windows-1251')

In [3]:
dp.head(n=1)

,Unnamed: 0,tk,pl,dia,hs,tc,cta,id_art,id_subsubfam,id_subfam,id_fam,id_famn,id_seccion,id_subagr,id_agr,vta,uni,id_artn,art_sp,fam_sp,fam_en
0,1242,120071109002055793,1,2007/11/09,0.505729,110000761,11000076,21895,101070640100,1010706401,10107064,10107064,10107,101,1,0.68,1.0,21895,MARAVILLA 500 G Store_Brand,PASTA ALIMENTICIA SE,pasta


In [4]:
user_dp = dp[['tc', 'art_sp']]

In [5]:
user_dp = user_dp.rename(columns={'tc':'user_id', 'art_sp': 'brand_info'})

In [6]:
user_dp.head()

,user_id,brand_info
0,110000761,MARAVILLA 500 G Store_Brand
1,110000761,FIDEO CABELLIN 500 G Store_Brand
2,28491841,SPAGUETTI Nє 5 500 G Brand_1
3,95931501,FIDEO FIDEUБ 500 Brand_7
4,93265591,MACARRONES GRATINAR 5 Brand_2


In [7]:
def split_brand(brand_name_data):
    return brand_name_data.split(' ')[-1]

In [8]:
user_dp['brand_name'] = user_dp.brand_info.apply(split_brand)

In [9]:
#user_dp['brand_name'] = user_dp.brand_info.apply(lambda x: x.split(' ')[-1])
# лучше использовать так

In [10]:
user_dp.head()

,user_id,brand_info,brand_name
0,110000761,MARAVILLA 500 G Store_Brand,Store_Brand
1,110000761,FIDEO CABELLIN 500 G Store_Brand,Store_Brand
2,28491841,SPAGUETTI Nє 5 500 G Brand_1,Brand_1
3,95931501,FIDEO FIDEUБ 500 Brand_7,Brand_7
4,93265591,MACARRONES GRATINAR 5 Brand_2,Brand_2


In [51]:
user_purchases = user_dp.groupby('user_id', as_index=False)\
                        .agg({'brand_name': 'count'})\
                        .rename(columns={'brand_name': 'purchases'})\
                        .query('purchases >= 5')

In [46]:
user_purchases.head()

,user_id,purchases
1,1031,6
2,4241,5
11,25971,7
14,40911,27
16,45181,5


In [58]:
user_uniq_brands = user_dp.groupby('user_id', as_index=False)\
    .agg({'brand_name': pd.Series.nunique})\
    .rename(columns={'brand_name': 'uniq_brands'})

In [59]:
user_uniq_brands

,user_id,uniq_brands
0,-1236394515,1
1,1031,2
2,4241,2
3,17311,1
4,17312,2
...,...,...
11759,1408817589,2
11760,1408825059,1
11761,1408832719,1
11762,1408840919,1


In [54]:
user_purchases.purchases.median()

7.0

In [14]:
user_purchases.describe()

,user_id,purchases
count,3.383000e+03,3383.000000
mean,6.421500e+07,9.320130
std,1.504830e+08,5.623993
min,1.031000e+03,5.000000
25%,8.871271e+06,6.000000
50%,2.842547e+07,7.000000
75%,8.542964e+07,11.000000
max,1.408810e+09,60.000000


In [30]:
user_dp.groupby(['user_id', 'brand_name'], as_index=False)\
        .agg({'brand_info': 'count'})\
        .query('user_id == 1031')

,user_id,brand_name,brand_info
1,1031,Brand_3,1
2,1031,Store_Brand,5


In [43]:
lovely_brand_purchases_df = user_dp.groupby(['user_id', 'brand_name'], as_index=False)\
        .agg({'brand_info': 'count'})\
        .sort_values(['user_id', 'brand_info'], ascending=[False, False])\
        .groupby('user_id')\
        .head(1)\
        .rename(columns={'brand_name': 'lovely_brand', 'brand_info': 'lovely_brand_purchases'})

In [60]:
lovely_brand_purchases_df

,user_id,lovely_brand,lovely_brand_purchases
18187,1408849249,Store_Brand,1
18186,1408840919,Store_Brand,1
18185,1408832719,Brand_4,3
18184,1408825059,Brand_1,1
18183,1408817589,Store_Brand,2
...,...,...,...
6,17312,Brand_1,1
5,17311,Brand_4,2
3,4241,Brand_4,3
2,1031,Store_Brand,5


In [63]:
loalty_df = user_purchases\
    .merge(user_uniq_brands, on='user_id')\
    .merge(lovely_brand_purchases_df, on='user_id')

In [66]:
loalty_df.head()

,user_id,purchases,uniq_brands,lovely_brand,lovely_brand_purchases
0,1031,6,2,Store_Brand,5
1,4241,5,2,Brand_4,3
2,25971,7,2,Store_Brand,5
3,40911,27,5,Brand_4,19
4,45181,5,4,Store_Brand,2


In [69]:
loayl_user = loalty_df[loalty_df.uniq_brands == 1]

In [72]:
loayl_user.head()

,user_id,purchases,uniq_brands,lovely_brand,lovely_brand_purchases
13,86281,14,1,Brand_4,14
18,94961,6,1,Brand_4,6
29,132061,9,1,Brand_4,9
30,134281,6,1,Brand_4,6
35,157311,12,1,Brand_4,12


In [74]:
loalty_df['loalty_score'] = loalty_df.lovely_brand_purchases / loalty_df.purchases

In [75]:
loalty_df.head()

,user_id,purchases,uniq_brands,lovely_brand,lovely_brand_purchases,loalty_score
0,1031,6,2,Store_Brand,5,0.833333
1,4241,5,2,Brand_4,3,0.600000
2,25971,7,2,Store_Brand,5,0.714286
3,40911,27,5,Brand_4,19,0.703704
4,45181,5,4,Store_Brand,2,0.400000
